In [12]:
import sys
import os
import pandas as pd
import math

In [3]:
align_path = "/scratch/indikar_root/indikar1/shared_data/single_cell/align_table/o4b03.GRCm39.align_table.parquet"

df = pd.read_parquet(align_path)
df.head()

,read_name,align_id,read_start,read_end,length_on_read,chrom,ref_start,ref_end,fragment_id,fragment_start,fragment_end,fragment_length,monomer_duplicate,is_mapped,mapping_quality
0,0043a53e-9c28-4353-ae6d-d84c5ed07f1b,180,0,153,153,NaN,-1,NaN,NaN,NaN,NaN,NaN,False,False,0
1,0056cbf0-fce9-4fa2-8614-7297e6073600,759,0,12,12,NaN,-1,NaN,NaN,NaN,NaN,NaN,False,False,0
2,00e0512d-39ab-599a-9cd5-f4ff5acb5c74,192,0,3,3,NaN,-1,NaN,NaN,NaN,NaN,NaN,False,False,0
3,01366e68-99e9-49e3-8cdf-fced6692a8d2,601,0,72,72,NaN,-1,NaN,NaN,NaN,NaN,NaN,False,False,0
4,015263d4-83a9-4966-ae0b-be7a80a539af,169,0,27,27,NaN,-1,NaN,NaN,NaN,NaN,NaN,False,False,0


In [28]:
def get_concatemers(df, mapq_threshold=1):
    """Filters a DataFrame for concatemers based on mapping criteria.

    Args:
        df (pandas.DataFrame): The DataFrame containing concatemer data.
        mapq_threshold (int, optional): Minimum mapping quality threshold. Defaults to 1.

    Returns:
        pandas.DataFrame: The filtered DataFrame containing only concatemers meeting the criteria.
    """
    
    # Filtering for mapped and high-quality concatemers
    filtered_df = df[
        (df['is_mapped']) & 
        (df['mapping_quality'] > mapq_threshold)
    ] 
    
    return filtered_df



print(f"{df.shape=}")    
pdf = get_concatemers(df)
print(f"{pdf.shape=}")    
    

df.shape=(861, 15)
pdf.shape=(43, 15)


In [29]:
res = []
for read_name, group in pdf.groupby('read_name'):
    order = len(group)
    n_direct = order - 1
    n_expanded = math.comb(order, 2)
    
    row = {
        'read_name' : read_name,
        'order' : order,
        'n_direct' : n_direct,
        'n_expanded' : n_expanded,
    }
    
    res.append(row)
    
res = pd.DataFrame(res)    
res.head()

,read_name,order,n_direct,n_expanded
0,1f5e4f67-0579-52b6-93c2-6e80da27aec1_1,3,2,3
1,243ee7e9-ffff-491f-bfd3-de987bbd6580_1,4,3,6
2,3b0e4908-3a58-4d76-b7ae-946bb396411d_1,2,1,1
3,608a6656-4510-4361-b1f2-b24f8fffac2e_1,3,2,3
4,786cd401-42cf-4c67-80d4-83d74d205827_2,5,4,10


In [30]:
res[['n_direct', 'n_expanded']].sum()

n_direct      27
n_expanded    46
dtype: int64

In [5]:
pdf.head()

,read_name,align_id,read_start,read_end,length_on_read,chrom,ref_start,ref_end,fragment_id,fragment_start,fragment_end,fragment_length,monomer_duplicate,is_mapped,mapping_quality
81,1f5e4f67-0579-52b6-93c2-6e80da27aec1_1,447,0,183,183,2,179654973,179655160.0,1798138.0,179654979.0,179655176.0,197.0,False,True,60
82,1f5e4f67-0579-52b6-93c2-6e80da27aec1_1,448,183,1004,821,2,179686556,179687306.0,1798282.0,179686558.0,179687311.0,753.0,False,True,60
83,1f5e4f67-0579-52b6-93c2-6e80da27aec1_1,449,1004,1222,218,4,37061664,37061908.0,2709203.0,37061670.0,37061914.0,244.0,False,True,40
100,243ee7e9-ffff-491f-bfd3-de987bbd6580_1,620,0,725,725,1,87553475,87554201.0,406014.0,87553467.0,87554203.0,736.0,False,True,60
101,243ee7e9-ffff-491f-bfd3-de987bbd6580_1,621,725,799,74,1,87554201,87554275.0,406015.0,87554203.0,87554277.0,74.0,False,True,43


In [6]:
df.columns

Index(['read_name', 'align_id', 'read_start', 'read_end', 'length_on_read',
       'chrom', 'ref_start', 'ref_end', 'fragment_id', 'fragment_start',
       'fragment_end', 'fragment_length', 'monomer_duplicate', 'is_mapped',
       'mapping_quality'],
      dtype='object')